<a href="https://colab.research.google.com/github/lzl1230/Federated-Learning-with-Local-Differential-Privacy/blob/master/fed_vcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Case

<img src="../images/use-case.png" alt="Drawing" style="width: 800px;"/>

## Demo Flow

1. Capture data.
- Construct feature matrix  
- Construct target vector
- Train/Test Split  
- Setup environment  
- Prepare federated data.
- Define a DL model.
- Define the training process.
- Define the validation process.
- Train model in federated way.
- Save, Load, Predict.

In [1]:
# !pip install syft==0.2.9 #最后运行的是这个
# !pip uninstall syft
# !pip install syft -f https://download.pytorch.org/whl/torch_stable.html
# !pip install torch
# !pip uninstall torch
# !pip install syft
# !pip install torchvision.models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Capture data and preprocess

In [72]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 10)
# We use the KDD CUP 1999 data (https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html)
# 41 column names can be found at https://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
# colnames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
#             'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
#             'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
#             'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
#             'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
#             'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
#             'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
#             'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
#             'dst_host_srv_rerror_rate']

# We take 10% of the original data which can be found at 
# http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
# We select the first 100K records from this data
df = pd.read_csv("/content/drive/My Drive/Colab_lzl/lzl/cic_ids_part.csv")
# df = pd.read_csv("/content/drive/My Drive/Colab_lzl/lzl/allFlows_0_small.csv")
df.head(3)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,...,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.005989,0.939505,0.000149,0.000056,0.005220,...,0.134167,0.006512,0.136667,0.128333,0
1,0.000000,0.947978,0.002616,0.000000,0.000000,...,0.101667,0.090546,0.173333,0.045875,0
2,0.082451,0.000834,0.000101,0.000000,0.000499,...,0.000000,0.000000,0.000000,0.000000,0


### Threat type distribution

In [73]:
import plotly.graph_objects as go
from collections import Counter

threat_count_dict = Counter(df["Label"])
print(threat_count_dict)
threat_types = list(threat_count_dict.keys())
print(threat_types)
threat_counts = [threat_count_dict[threat_type] for threat_type in threat_types]
print("Total distinct number of threat types : ",len(threat_types))
# fig = go.Figure([go.Bar(x=threat_types, y=threat_counts,text=threat_counts,textposition='auto')])
# fig.show()

Counter({0: 189038, 1: 1966})
[0, 1]
Total distinct number of threat types :  2


### 2. Construct the feature matrix

In [41]:
# 34 numerical columns are considered for training
numerical_colmanes = [' Destination Port', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
                      ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max',
                      ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
                      ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total',
                      ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min',
                      'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
                      ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
                      ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',' Fwd Header Length',' Bwd Header Length','Fwd Packets/s',' Bwd Packets/s',' Min Packet Length',' Max Packet Length',' Packet Length Mean',
                      ' Packet Length Std',' Packet Length Variance','FIN Flag Count',' SYN Flag Count',
                      ' RST Flag Count',' PSH Flag Count',' ACK Flag Count',' URG Flag Count',' CWE Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size',' Avg Fwd Segment Size',' Fwd Header Length','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',
                      ' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets',' Subflow Fwd Bytes',' Subflow Bwd Packets',' Subflow Bwd Bytes','Init_Win_bytes_forward',' Init_Win_bytes_backward',' act_data_pkt_fwd',
                      ' min_seg_size_forward','Active Mean',' Active Std',' Active Max',' Active Min','Idle Mean',' Idle Std',' Idle Max',' Idle Min']
numerical_df = df[numerical_colmanes].copy()
# Lets remove the numerical columns with constant value
numerical_df = numerical_df.loc[:, (numerical_df != numerical_df.iloc[0]).any()]   #这个注释掉了
# lets scale the values for each column from [0,1]
# N.B. we dont have any negative values]
final_df = numerical_df/numerical_df.max()
X = final_df.values
# final dataframe has 67 features
print("Shape of feature matrix : ",X.shape)

Shape of feature matrix :  (191004, 67)


### 3. Construct the target  vector

In [42]:
from sklearn.preprocessing import LabelEncoder

threat_types = df["Label"].values
encoder = LabelEncoder()
# use LabelEncoder to encode the threat types in numeric values
y = encoder.fit_transform(threat_types)
print("Shape of target vector : ",y.shape)

Shape of target vector :  (191004,)


### 4. Train/Test Split

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42, stratify=y)
print(X_train.shape[1])
print("Number of records in training data : ", X_train.shape[0])
print("Number of records in test data : ", X_test.shape[0])
print("Total distinct number of threat types in training data : ",len(set(y_train)))
print("Total distinct number of threat types in test data : ",len(set(y_test)))

67
Number of records in training data :  133702
Number of records in test data :  57302
Total distinct number of threat types in training data :  2
Total distinct number of threat types in test data :  2


### 5. Lets set up the environment for federated learning

In [44]:
#capture
import torch
import syft as sy

# Hook PyTorch ie add extra functionalities to support Federated Learning
hook = sy.TorchHook(torch)
# Sets the seed for generating random numbers.
torch.manual_seed(1)
# Select CPU computation, in case you want GPU use "cuda" instead
device = torch.device("cpu")
# Data will be distributed among these VirtualWorkers.
# Remote training of the model will happen here.
gatway1 = sy.VirtualWorker(hook, id="gatway1")
gatway2 = sy.VirtualWorker(hook, id="gatway2")

### 5. Lets set the training params

In [45]:
import numpy as np

# Number of times we want to iterate over whole training data
BATCH_SIZE = 500 #1000
EPOCHS = 2  #2
LOG_INTERVAL = 5
lr = 0.2  #0.01

n_feature = X_train.shape[1]
n_class = np.unique(y_train).shape[0]



print("Number of training features : ",n_feature)
print("Number of training classes : ",n_class)

Number of training features :  67
Number of training classes :  2


### 6. Prepare Federated data and distribute across the gateways

In [46]:
# Create pytorch tensor from X_train,y_train,X_test,y_test
train_inputs = torch.tensor(X_train,dtype=torch.float).tag("#iot", "#network","#data","#train")
train_labels = torch.tensor(y_train).tag("#iot", "#network","#target","#train")
test_inputs = torch.tensor(X_test,dtype=torch.float).tag("#iot", "#network","#data","#test")
test_labels = torch.tensor(y_test).tag("#iot", "#network","#target","#test")
# print(train_labels)

# Send the training and test data to the gatways in equal proportion.
train_idx = int(len(train_labels)/2)
test_idx = int(len(test_labels)/2)
gatway1_train_dataset = sy.BaseDataset(train_inputs[:train_idx], train_labels[:train_idx]).send(gatway1)
gatway2_train_dataset = sy.BaseDataset(train_inputs[train_idx:], train_labels[train_idx:]).send(gatway2)
gatway1_test_dataset = sy.BaseDataset(test_inputs[:test_idx], test_labels[:test_idx]).send(gatway1)
gatway2_test_dataset = sy.BaseDataset(test_inputs[test_idx:], test_labels[test_idx:]).send(gatway2)

# Create federated datasets, an extension of Pytorch TensorDataset class
federated_train_dataset = sy.FederatedDataset([gatway1_train_dataset, gatway2_train_dataset])
federated_test_dataset = sy.FederatedDataset([gatway1_test_dataset, gatway2_test_dataset])

# Create federated dataloaders, an extension of Pytorch DataLoader class
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
federated_test_loader = sy.FederatedDataLoader(federated_test_dataset, shuffle=False, batch_size=BATCH_SIZE)

### 7. Lets define a simple Logistic Regression Model in Pytorch

In [47]:
import sys
import torch.nn as nn
import math
from torch.autograd import Variable
import torch.nn.functional as F

In [34]:
class SparseAutoEncoder(nn.Module): #普通的自动编码器
  def __init__(self, feature_size, hidden_size):
    super(SparseAutoEncoder, self).__init__()
    self.feature_size = feature_size
    self.hidden_size = hidden_size

    # Sigmoid layer
    self.sigmoid = nn.Sigmoid()

    # Encoder layers
    self.layer1 = nn.Linear(feature_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, feature_size)

  # Feedforward
  def forward(self, x):
    x = self.sigmoid(self.layer1(x))
    x = self.sigmoid(self.layer2(x))
    return x

In [35]:
model_lzl = SparseAutoEncoder(n_feature,3)
# t = torch.ones(10, 3, 4) #batch_size, seq_size, input_size

print(model_lzl)

SparseAutoEncoder(
  (sigmoid): Sigmoid()
  (layer1): Linear(in_features=62, out_features=3, bias=True)
  (layer2): Linear(in_features=3, out_features=62, bias=True)
)


In [ ]:
n_input = n_feature # Must match the shape of the input features
n_hidden1 = 12 # Number of neurons in the 1st hidden layer
n_hidden2 = 6 # Number of neurons in the 2nd hidden layer
n_output = 3 # Number of output units (for example 1 for binary classification)

In [ ]:
class Network(nn.Module): #CNN
  def __init__(self):
    super(Network, self).__init__()
    # Inputs to the 1st hidden layer linear transformation
    self.hidden1 = nn.Linear(n_input,n_hidden1)
    # Inputs to the 2nd hidden layer linear transformation
    self.hidden2 = nn.Linear(n_hidden1,n_hidden2)
    # Activation function for the hidden layers' output - ReLU
    self.relu = nn.ReLU()
    # Output layer linear transformation
    self.output = nn.Linear(n_hidden2, n_output)
    # Activation function for the output layer - sigmoid
    self.sigmoid = nn.Sigmoid()
  def forward(self, X, **kwargs):
    # Passes the input tensor through each of the defined operations
    X = self.hidden1(X)
    X = self.relu(X)
    X = self.hidden2(X)
    X = self.relu(X)
    X = self.output(X)
    X = self.sigmoid(X)
    return X

In [ ]:
# model = Network()
# print(model)

Network(
  (hidden1): Linear(in_features=67, out_features=8, bias=True)
  (hidden2): Linear(in_features=8, out_features=4, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [31]:
# import torch.nn as nn#简单的逻辑回归
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        """
        input_dim: number of input features.
        output_dim: number of labels.
        """
        super(Net, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        outputs = self.linear(x)
        return outputs

参数：
        # input_dim 输入特征维度d_input
        # hidden_dim 隐藏层的大小
        # output_dim 输出层的大小（分类的类别数）
        # num_layers LSTM隐藏层的层数
        # biFlag 是否使用双向

In [48]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size, center_size):
    super(Encoder, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size) 
    self.l2 = nn.Linear(hidden_size, center_size)  
    self.acti = nn.ELU()
    
  def forward(self, x):
    out = self.l1(x)
    out = self.acti(out)
    out = self.l2(out)
    return out

class Decoder(nn.Module):   
  def __init__(self, output_size, hidden_size, center_size):
    super(Decoder, self).__init__()
    self.l1 = nn.Linear(center_size, hidden_size) 
    self.l2 = nn.Linear(hidden_size, output_size)  
    self.acti = nn.ELU()
    
  def forward(self, x):
    out = self.l1(x)
    out = self.acti(out)
    out = self.l2(out)
    return out

In [49]:
class RecurrentAutoencoder(nn.Module):
  def __init__(self, input_size, output_size, hidden_size, center_size):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(input_size, hidden_size, center_size).to(device)
    self.decoder = Decoder(output_size, hidden_size, center_size).to(device)
    print(self.encoder)
    print(self.decoder)

  def forward(self, x):
    encoded = self.encoder(x)
    dencoded = self.decoder(encoded)
    return dencoded

In [50]:
input_size = X_train.shape[1]
output_size = X_train.shape[1]

# hidden_size = 128
# center_size = 256

hidden_size = 7
center_size = 2

In [51]:
model = RecurrentAutoencoder(input_size, output_size, hidden_size, center_size).to(device)
print(model)

Encoder(
  (l1): Linear(in_features=67, out_features=7, bias=True)
  (l2): Linear(in_features=7, out_features=2, bias=True)
  (acti): ELU(alpha=1.0)
)
Decoder(
  (l1): Linear(in_features=2, out_features=7, bias=True)
  (l2): Linear(in_features=7, out_features=67, bias=True)
  (acti): ELU(alpha=1.0)
)
RecurrentAutoencoder(
  (encoder): Encoder(
    (l1): Linear(in_features=67, out_features=7, bias=True)
    (l2): Linear(in_features=7, out_features=2, bias=True)
    (acti): ELU(alpha=1.0)
  )
  (decoder): Decoder(
    (l1): Linear(in_features=2, out_features=7, bias=True)
    (l2): Linear(in_features=7, out_features=67, bias=True)
    (acti): ELU(alpha=1.0)
  )
)


### 8. Lets define the training process 

In [52]:
import torch.nn.functional as F

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()
    # Iterate through each gateway's dataset
    for idx, (data, target) in enumerate(federated_train_loader):
        batch_idx = idx+1
        # Send the model to the right gateway
        model.send(data.location)
        # Move the data and target labels to the device (cpu/gpu) for computation
        data, target = data.to(device), target.to(device)
        # Clear previous gradients (if they exist)
        optimizer.zero_grad()
        # Make a prediction
        output = model(data)
        # Calculate the cross entropy loss [We are doing classification]
        loss = F.cross_entropy(output, target)
        # Calculate the gradients
        loss.backward()
        # Update the model weights
        optimizer.step()
        # Get the model back from the gateway
        model.get()
        if batch_idx==len(federated_train_loader) or (batch_idx!=0 and batch_idx % LOG_INTERVAL == 0):
            # get the loss back
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * BATCH_SIZE, len(federated_train_loader) * BATCH_SIZE,
                100. * batch_idx / len(federated_train_loader), loss.item()))

### 9. Lets define the validation process

In [53]:
import torch.nn.functional as F

def test(model, device, federated_test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(federated_test_loader):
            # Send the model to the right gateway
            model.send(data.location)
            # Move the data and target labels to the device (cpu/gpu) for computation
            data, target = data.to(device), target.to(device)
            # Make a prediction
            output = model(data)
            # Get the model back from the gateway
            model.get()
            # Calculate the cross entropy loss
            loss = F.cross_entropy(output, target)
            # Get the index of the max log-probability 
            pred = output.argmax(1, keepdim=True)
            # Get the number of instances correctly predicted
            correct += pred.eq(target.view_as(pred)).sum().get()
                
    # get the loss back
    loss = loss.get()
    print('Test set: Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss.item(), correct, len(federated_test_loader.federated_dataset),
        100. * correct / len(federated_test_loader.federated_dataset)))

### 10. Lets train the model in Federated way

In [71]:
#%%time
import torch.optim as optim

# Initialize the model
# model = Net(n_feature,n_class) #==============================================================模型在这输入
# model = Network()
# model = model_lzl
# model = model


#Initialize the SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=lr)# lr学习率

for epoch in range(1, EPOCHS + 1):
    # Train on the training data in a federated way
    train(model, device, federated_train_loader, optimizer, epoch)
    # Check the test accuracy on unseen test data in a federated way
    test(model, device, federated_test_loader)

AttributeError: ignored

### 11. Save, reload and use the model to predict a network traffic data。  到这里我就不太需要了

In [ ]:
# Save the model
torch.save(model.state_dict(), "/content/drive/My Drive/Colab_lzl/lzl/binaize-threat-model.pt")
# Reload the model in a new model object
model_new = Net(n_feature,n_class)
model_new.load_state_dict(torch.load("/content/drive/My Drive/Colab_lzl/lzl/binaize-threat-model.pt"))
model_new.eval()

Net(
  (linear): Linear(in_features=67, out_features=2, bias=True)
)

In [ ]:
# Take the 122th record from the test data
idx = 122
data = test_inputs[idx]
pred = model_new(data)
pred_label = int(pred.argmax().data.cpu().numpy())
pred_threat = encoder.inverse_transform([pred_label])[0]
print("Predicted threat type : ", pred_threat)
actual_label = int(test_labels[idx].data.cpu().numpy())
actual_threat = encoder.inverse_transform([actual_label])[0]
print("Actual threat type : ", actual_threat)

Predicted threat type :  0
Actual threat type :  0


In [ ]:
# Take the 159th record from the test data
idx = 159
data = test_inputs[idx]
pred = model_new(data)
pred_label = int(pred.argmax().data.cpu().numpy())
pred_threat = encoder.inverse_transform([pred_label])[0]
print("Predicted threat type : ", pred_threat)
actual_label = int(test_labels[idx].data.cpu().numpy())
actual_threat = encoder.inverse_transform([actual_label])[0]
print("Actual threat type : ", actual_threat)

Predicted threat type :  0
Actual threat type :  0
